In [2]:
%conda install pandas

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /Users/jingzhao/anaconda3

  added / updated specs:
    - pandas


The following packages will be UPDATED:

  pandas                              2.0.3-py311hdb55bb0_0 --> 2.1.4-py311hdb55bb0_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [3]:
%conda install statsmodels

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [4]:
%conda install scikit-learn

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [85]:
import pandas as pd
industry_portfolio_data = pd.read_csv('/Users/jingzhao/Desktop/FE PW1/Data/17 Industry Portfolios.CSV')
financial_data = pd.read_csv('/Users/jingzhao/Desktop/FE PW1/Data/Financial Data.CSV')

In [86]:
# HML: Regressor Selection
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Data Construction
merged_data = pd.merge(industry_portfolio_data, financial_data[['Date', 'HML']], on='Date')
missing_values = merged_data.isnull().sum()

# Preparing the data for the model
X = merged_data.drop(['Date', 'HML'], axis=1)
y = merged_data['HML']

# Random Forest Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Feature importances
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

In [87]:
# HML: Linear Regression
from sklearn.linear_model import LinearRegression
selected_features = feature_importances['Feature'].tolist()

# Preparing the data with selected features
X_selected = merged_data[selected_features]

# Splitting the data into training and testing sets for the selected features
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(
    X_selected, y, test_size=0.2, random_state=42)

# Creating and fitting the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_selected, y_train_selected)

# Making predictions and evaluating the model
y_pred_selected = lr_model.predict(X_test_selected)
mse_selected = mean_squared_error(y_test_selected, y_pred_selected)
rmse_selected = np.sqrt(mse_selected)

# Output the model performance
mse_selected, rmse_selected

import statsmodels.api as sm

# Creating and fitting the OLS model
## NOTE: Covariance type is changed to hetero-robust instead of homo
ols_model = sm.OLS(y_train_selected, X_train_selected).fit(cov_type='HC3')

# Printing the summary of the OLS model
ols_summary = ols_model.summary()
ols_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    HML   R-squared (uncentered):                   0.568
Model:                            OLS   Adj. R-squared (uncentered):              0.559
Method:                 Least Squares   F-statistic:                              23.63
Date:                Sat, 06 Jan 2024   Prob (F-statistic):                    4.89e-61
Time:                        14:14:15   Log-Likelihood:                         -2115.8
No. Observations:                 930   AIC:                                      4266.
Df Residuals:                     913   BIC:                                      4348.
Df Model:                          17                                                  
Covariance Type:                  HC3                                                  
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Trans          0.3061      0.055      5.575      0.000       0.198       0.414
Steel          0.1786      0.034      5.249      0.000       0.112       0.245
Machn         -0.2467      0.035     -7.058      0.000      -0.315      -0.178
Finan          0.2965      0.051      5.802      0.000       0.196       0.397
Other         -0.4167      0.066     -6.299      0.000      -0.546      -0.287
Oil            0.0688      0.031      2.192      0.028       0.007       0.130
Cnsum         -0.0969      0.045     -2.129      0.033      -0.186      -0.008
Rtail         -0.1563      0.041     -3.811      0.000      -0.237      -0.076
Utils          0.1072      0.035      3.094      0.002       0.039       0.175
Durbl          0.1272      0.046      2.789      0.005       0.038       0.217
FabPr         -0.1727      0.037     -4.636      0.000      -0.246      -0.100
Clths          0.0200      0.065      0.307      0.759      -0.107       0.147
Chems         -0.0708      0.035     -2.011      0.044      -0.140      -0.002
Mines         -0.0428      0.024     -1.812      0.070      -0.089       0.003
Cars           0.0581      0.025      2.303      0.021       0.009       0.108
Food           0.0279      0.056      0.498      0.618      -0.082       0.138
Cnstr         -0.0141      0.045     -0.312      0.755      -0.103       0.074
==============================================================================
Omnibus:                       61.314   Durbin-Watson:                   1.912
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              172.523
Skew:                           0.309   Prob(JB):                     3.44e-38
Kurtosis:                       5.018   Cond. No.                         14.2
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors are heteroscedasticity robust (HC3)
"""

In [88]:
# HML Rolling Regression
from datetime import timedelta
from sklearn.linear_model import LinearRegression

merged_data.columns = merged_data.columns.str.strip()
def rolling_regression(data, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')

    # List all 17 industry portfolios as regressors
    selected_features = ['Food', 'Mines', 'Oil', 'Clths', 'Durbl', 'Chems', 'Cnsum', 
                         'Cnstr', 'Steel', 'FabPr', 'Machn', 'Cars', 'Trans', 'Utils', 
                         'Rtail', 'Finan', 'Other']

    unique_years = data['Date'].dt.year.unique()
    
    results = []
    start_year = unique_years[0]

    # Perform rolling regression
    while start_year + train_years + test_years <= unique_years[-1]:
        # Define training and testing periods
        train_start = pd.Timestamp(year=start_year, month=1, day=1)
        train_end = train_start + timedelta(days=365 * train_years)
        test_end = train_end + timedelta(days=365 * test_years)

        # Subset the data for training and testing
        train_data = data[(data['Date'] >= train_start) & (data['Date'] < train_end)]
        test_data = data[(data['Date'] >= train_end) & (data['Date'] < test_end)]

        # Fit the model
        X_train = train_data[selected_features]
        y_train = train_data['HML']
        model = LinearRegression().fit(X_train, y_train)

        # Predict on test data
        X_test = test_data[selected_features]
        y_pred = model.predict(X_test)

        # Collect coefficients and predictions
        coefficients = model.coef_
        results.append({
            'train_start': train_start,
            'train_end': train_end,
            'test_end': test_end,
            'coefficients': coefficients,
            'predicted_HML': y_pred
        })

        # Move to the next period
        start_year += test_years

    return pd.DataFrame(results)

# Rolling regression for each scenario
results_5_year = rolling_regression(merged_data, 5, 5)
results_10_year = rolling_regression(merged_data, 10, 5)
results_20_year = rolling_regression(merged_data, 20, 5)

def extract_betas_and_predictions(results, industry_portfolios):
    betas = []
    predicted_HML = []

    for index, row in results.iterrows():
        beta_dict = {
            'train_start': row['train_start'],
            'train_end': row['train_end'],
            'test_end': row['test_end']
        }
        
        for i, industry in enumerate(industry_portfolios):
            beta_dict[f'{industry}_beta'] = row['coefficients'][i]
        
        betas.append(beta_dict)
        predicted_HML.extend(row['predicted_HML'])

    betas_df = pd.DataFrame(betas)
    predicted_HML_df = pd.DataFrame({'predicted_HML': predicted_HML})

    return betas_df, predicted_HML_df

industry_portfolios = ['Food', 'Mines', 'Oil', 'Clths', 'Durbl', 'Chems', 'Cnsum', 
                       'Cnstr', 'Steel', 'FabPr', 'Machn', 'Cars', 'Trans', 'Utils', 
                       'Rtail', 'Finan', 'Other']

# Extracting for each time scheme
betas_5_year, predicted_HML_5_year = extract_betas_and_predictions(results_5_year, industry_portfolios)
betas_10_year, predicted_HML_10_year = extract_betas_and_predictions(results_10_year, industry_portfolios)
betas_20_year, predicted_HML_20_year = extract_betas_and_predictions(results_20_year, industry_portfolios)
betas_5_year, predicted_HML_5_year

(   train_start  train_end   test_end  Food_beta  Mines_beta  Oil_beta  \
 0   1927-01-01 1931-12-31 1936-12-29   0.225737   -0.019864  0.175299   
 1   1932-01-01 1936-12-30 1941-12-29  -0.192584   -0.054609  0.119764   
 2   1937-01-01 1941-12-31 1946-12-30   0.131484   -0.039389  0.301208   
 3   1942-01-01 1946-12-31 1951-12-30  -0.118150    0.063429  0.155608   
 4   1947-01-01 1951-12-31 1956-12-29   0.107808    0.112041  0.144859   
 5   1952-01-01 1956-12-30 1961-12-29   0.292075    0.010123 -0.023117   
 6   1957-01-01 1961-12-31 1966-12-30  -0.204629    0.007674  0.025773   
 7   1962-01-01 1966-12-31 1971-12-30   0.285842   -0.026342  0.327466   
 8   1967-01-01 1971-12-31 1976-12-29  -0.189294   -0.120569  0.129331   
 9   1972-01-01 1976-12-30 1981-12-29  -0.051847    0.009850 -0.286878   
 10  1977-01-01 1981-12-31 1986-12-30   0.179347   -0.193840 -0.095422   
 11  1982-01-01 1986-12-31 1991-12-30  -0.059533    0.012330  0.094594   
 12  1987-01-01 1991-12-31 1996-12-29 

In [89]:
# Residual
def calculate_residuals(data, predicted_HML, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()
    residuals = []

    start_year = unique_years[0]
    predicted_index = 0

    while start_year + train_years + test_years <= unique_years[-1]:
        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_HML = test_data['HML'].values

        # Calculate residuals
        predicted_HML_values = predicted_HML['predicted_HML'].iloc[predicted_index:predicted_index + len(actual_HML)]
        residual = actual_HML - predicted_HML_values
        residuals.extend(residual)

        # Update indices
        predicted_index += len(actual_HML)
        start_year += test_years

    return pd.DataFrame({'residuals': residuals})

# Calculate residuals for each rolling scheme
residuals_5_year = calculate_residuals(merged_data, predicted_HML_5_year, 5, 5)
residuals_10_year = calculate_residuals(merged_data, predicted_HML_10_year, 10, 5)
residuals_20_year = calculate_residuals(merged_data, predicted_HML_20_year, 20, 5)

residual_summary_5_year = residuals_5_year.describe()
residual_summary_10_year = residuals_10_year.describe()
residual_summary_20_year = residuals_20_year.describe()

residual_summary_5_year, residual_summary_10_year, residual_summary_20_year

(         residuals
 count  1080.000000
 mean     -0.144324
 std       2.627564
 min     -13.519619
 25%      -1.541285
 50%      -0.288517
 75%       1.280076
 max      18.252588,
          residuals
 count  1020.000000
 mean     -0.122225
 std       2.129064
 min     -10.531912
 25%      -1.355265
 50%      -0.202789
 75%       1.045957
 max       9.826449,
         residuals
 count  900.000000
 mean    -0.173789
 std      2.025429
 min     -8.039950
 25%     -1.424272
 50%     -0.282700
 75%      0.934058
 max      6.785557)

In [90]:
# SSE for each time scheme
sse_5_year = np.sum(residuals_5_year['residuals'] ** 2)
sse_10_year = np.sum(residuals_10_year['residuals'] ** 2)
sse_20_year = np.sum(residuals_20_year['residuals'] ** 2)

sse_5_year, sse_10_year, sse_20_year

(7472.013824415835, 4634.276112332735, 3715.206031934155)

In [91]:
# HML OOR2
def calculate_out_of_sample_r_squared(data, predicted_HML, residuals, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()

    total_sum_squares = 0
    residual_sum_squares = 0
    predicted_index = 0

    for start_year in unique_years:
        if start_year + train_years + test_years > unique_years[-1]:
            break

        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_HML = test_data['HML'].values

        # Calculate total sum of squares and residual sum of squares
        mean_actual_HML = np.mean(actual_HML)
        total_sum_squares += np.sum((actual_HML - mean_actual_HML) ** 2)
        residual_sum_squares += np.sum(residuals['residuals'].iloc[predicted_index:predicted_index + len(actual_HML)] ** 2)

        # Update indices
        predicted_index += len(actual_HML)

    return 1 - (residual_sum_squares / total_sum_squares)

# Calculate out-of-sample R-squared for each rolling scheme
r_squared_5_year = calculate_out_of_sample_r_squared(merged_data, predicted_HML_5_year, residuals_5_year, 5, 5)
r_squared_10_year = calculate_out_of_sample_r_squared(merged_data, predicted_HML_10_year, residuals_10_year, 10, 5)
r_squared_20_year = calculate_out_of_sample_r_squared(merged_data, predicted_HML_20_year, residuals_20_year, 20, 5)
r_squared_5_year, r_squared_10_year, r_squared_20_year

(0.84759175456176, 0.8779221245139004, 0.8818784218952873)

In [94]:
# MOM Get Regressors
merged_data = pd.merge(industry_portfolio_data, financial_data[['Date', 'MOM']], on='Date')
missing_values = merged_data.isnull().sum()
X = merged_data.drop(['Date', 'MOM'], axis=1)
y = merged_data['MOM']

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating and fitting the Random Forest model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Making predictions and evaluating the model
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Output the model performance and the missing values information
mse, rmse, missing_values
# Getting feature importances from the Random Forest model
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

In [95]:
# MOM: Linear Regression
selected_features = top_5_portfolios['Feature'].tolist()
X_selected = merged_data[selected_features]

# Splitting data
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(
    X_selected, y, test_size=0.2, random_state=42)

# Creating and fitting the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_selected, y_train_selected)

# Making predictions and evaluating the model
y_pred_selected = lr_model.predict(X_test_selected)
mse_selected = mean_squared_error(y_test_selected, y_pred_selected)
rmse_selected = np.sqrt(mse_selected)

# Output the model performance
mse_selected, rmse_selected

import statsmodels.api as sm

# Creating and fitting the OLS model
ols_model = sm.OLS(y_train_selected, X_train_selected).fit()

# Printing the summary of the OLS model
ols_summary = ols_model.summary()
ols_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    MOM   R-squared (uncentered):                   0.217
Model:                            OLS   Adj. R-squared (uncentered):              0.213
Method:                 Least Squares   F-statistic:                              51.23
Date:                Sat, 06 Jan 2024   Prob (F-statistic):                    6.09e-47
Time:                        14:24:23   Log-Likelihood:                         -2642.8
No. Observations:                 930   AIC:                                      5296.
Df Residuals:                     925   BIC:                                      5320.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Trans         -0.1659      0.042     -3.956      0.000      -0.248      -0.084
FabPr          0.1316      0.044      3.018      0.003       0.046       0.217
Machn          0.1856      0.039      4.728      0.000       0.109       0.263
Finan         -0.2328      0.041     -5.715      0.000      -0.313      -0.153
Steel         -0.1492      0.030     -4.902      0.000      -0.209      -0.089
==============================================================================
Omnibus:                      311.184   Durbin-Watson:                   1.775
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2831.555
Skew:                          -1.258   Prob(JB):                         0.00
Kurtosis:                      11.169   Cond. No.                         5.56
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [96]:
# MOM Rolling Regression
from datetime import timedelta
from sklearn.linear_model import LinearRegression

merged_data.columns = merged_data.columns.str.strip()
def rolling_regression(data, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')

    # List all 17 industry portfolios as regressors
    selected_features = ['Food', 'Mines', 'Oil', 'Clths', 'Durbl', 'Chems', 'Cnsum', 
                         'Cnstr', 'Steel', 'FabPr', 'Machn', 'Cars', 'Trans', 'Utils', 
                         'Rtail', 'Finan', 'Other']

    unique_years = data['Date'].dt.year.unique()
    
    results = []
    start_year = unique_years[0]

    # Perform rolling regression
    while start_year + train_years + test_years <= unique_years[-1]:
        # Define training and testing periods
        train_start = pd.Timestamp(year=start_year, month=1, day=1)
        train_end = train_start + timedelta(days=365 * train_years)
        test_end = train_end + timedelta(days=365 * test_years)

        # Subset the data for training and testing
        train_data = data[(data['Date'] >= train_start) & (data['Date'] < train_end)]
        test_data = data[(data['Date'] >= train_end) & (data['Date'] < test_end)]

        # Fit the model
        X_train = train_data[selected_features]
        y_train = train_data['MOM']
        model = LinearRegression().fit(X_train, y_train)

        # Predict on test data
        X_test = test_data[selected_features]
        y_pred = model.predict(X_test)

        # Collect coefficients and predictions
        coefficients = model.coef_
        results.append({
            'train_start': train_start,
            'train_end': train_end,
            'test_end': test_end,
            'coefficients': coefficients,
            'predicted_MOM': y_pred
        })

        # Move to the next period
        start_year += test_years

    return pd.DataFrame(results)

# Rolling regression for each scenario
results_5_year = rolling_regression(merged_data, 5, 5)
results_10_year = rolling_regression(merged_data, 10, 5)
results_20_year = rolling_regression(merged_data, 20, 5)

def extract_betas_and_predictions(results, industry_portfolios):
    betas = []
    predicted_MOM = []

    for index, row in results.iterrows():
        beta_dict = {
            'train_start': row['train_start'],
            'train_end': row['train_end'],
            'test_end': row['test_end']
        }
        
        for i, industry in enumerate(industry_portfolios):
            beta_dict[f'{industry}_beta'] = row['coefficients'][i]
        
        betas.append(beta_dict)
        predicted_MOM.extend(row['predicted_MOM'])

    betas_df = pd.DataFrame(betas)
    predicted_MOM_df = pd.DataFrame({'predicted_MOM': predicted_MOM})

    return betas_df, predicted_MOM_df

industry_portfolios = ['Food', 'Mines', 'Oil', 'Clths', 'Durbl', 'Chems', 'Cnsum', 
                       'Cnstr', 'Steel', 'FabPr', 'Machn', 'Cars', 'Trans', 'Utils', 
                       'Rtail', 'Finan', 'Other']

# Extracting for each time scheme
betas_5_year, predicted_MOM_5_year = extract_betas_and_predictions(results_5_year, industry_portfolios)
betas_10_year, predicted_MOM_10_year = extract_betas_and_predictions(results_10_year, industry_portfolios)
betas_20_year, predicted_MOM_20_year = extract_betas_and_predictions(results_20_year, industry_portfolios)
betas_5_year, predicted_MOM_5_year

(   train_start  train_end   test_end  Food_beta  Mines_beta  Oil_beta  \
 0   1927-01-01 1931-12-31 1936-12-29   0.215790    0.136198 -0.164491   
 1   1932-01-01 1936-12-30 1941-12-29   0.309771   -0.445477  0.240749   
 2   1937-01-01 1941-12-31 1946-12-30   1.220014   -0.250833 -0.039079   
 3   1942-01-01 1946-12-31 1951-12-30  -0.177578   -0.287085  0.236538   
 4   1947-01-01 1951-12-31 1956-12-29  -0.249570    0.193939  0.287711   
 5   1952-01-01 1956-12-30 1961-12-29  -0.114166   -0.142376  0.028684   
 6   1957-01-01 1961-12-31 1966-12-30   0.567789   -0.278017  0.073915   
 7   1962-01-01 1966-12-31 1971-12-30   0.035956    0.051618  0.070730   
 8   1967-01-01 1971-12-31 1976-12-29   1.607385   -0.390310  0.298016   
 9   1972-01-01 1976-12-30 1981-12-29  -0.343581   -0.067146  0.066215   
 10  1977-01-01 1981-12-31 1986-12-30  -0.190772    0.130881  0.241090   
 11  1982-01-01 1986-12-31 1991-12-30   0.287754   -0.021651 -0.226516   
 12  1987-01-01 1991-12-31 1996-12-29 

In [97]:
# MOM Residual
def calculate_residuals(data, predicted_MOM, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()
    residuals = []

    start_year = unique_years[0]
    predicted_index = 0

    while start_year + train_years + test_years <= unique_years[-1]:
        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_MOM = test_data['MOM'].values

        # Calculate residuals
        predicted_MOM_values = predicted_MOM['predicted_MOM'].iloc[predicted_index:predicted_index + len(actual_MOM)]
        residual = actual_MOM - predicted_MOM_values
        residuals.extend(residual)

        # Update indices
        predicted_index += len(actual_MOM)
        start_year += test_years

    return pd.DataFrame({'residuals': residuals})

# Calculate residuals for each rolling scheme
residuals_5_year = calculate_residuals(merged_data, predicted_MOM_5_year, 5, 5)
residuals_10_year = calculate_residuals(merged_data, predicted_MOM_10_year, 10, 5)
residuals_20_year = calculate_residuals(merged_data, predicted_MOM_20_year, 20, 5)

residuals_5_year.head(), residuals_10_year.head(), residuals_20_year.head()


(   residuals
 0 -10.720764
 1   9.148963
 2   2.763317
 3   2.455588
 4  10.264896,
    residuals
 0  -6.637947
 1   8.203690
 2   4.258663
 3 -10.735683
 4  -4.070594,
    residuals
 0  -5.006253
 1  -2.653277
 2   2.645168
 3   2.858686
 4   2.222193)

In [64]:
# Residual Summary
residual_summary_5_year = residuals_5_year.describe()
residual_summary_10_year = residuals_10_year.describe()
residual_summary_20_year = residuals_20_year.describe()

residual_summary_5_year, residual_summary_10_year, residual_summary_20_year

(         residuals
 count  1080.000000
 mean     -0.031597
 std       2.475316
 min     -13.710342
 25%      -1.332233
 50%      -0.075307
 75%       1.360415
 max      10.027822,
          residuals
 count  1020.000000
 mean     -0.098707
 std       2.148893
 min     -13.030419
 25%      -1.326080
 50%      -0.103802
 75%       1.141364
 max       9.186277,
         residuals
 count  900.000000
 mean    -0.130205
 std      2.086146
 min     -8.466385
 25%     -1.372565
 50%     -0.163566
 75%      1.103427
 max      8.381515)

In [99]:
# SSE for each time scheme
sse_5_year = np.sum(residuals_5_year['residuals'] ** 2)
sse_10_year = np.sum(residuals_10_year['residuals'] ** 2)
sse_20_year = np.sum(residuals_20_year['residuals'] ** 2)

sse_5_year, sse_10_year, sse_20_year

(28882.64287420492, 20290.76153222713, 15406.472903182286)

In [100]:
# MOM OOR2
def calculate_out_of_sample_r_squared(data, predicted_MOM, residuals, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()

    total_sum_squares = 0
    residual_sum_squares = 0
    predicted_index = 0

    for start_year in unique_years:
        if start_year + train_years + test_years > unique_years[-1]:
            break

        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_MOM = test_data['MOM'].values

        # Calculate total sum of squares and residual sum of squares
        mean_actual_MOM = np.mean(actual_MOM)
        total_sum_squares += np.sum((actual_MOM - mean_actual_MOM) ** 2)
        residual_sum_squares += np.sum(residuals['residuals'].iloc[predicted_index:predicted_index + len(actual_MOM)] ** 2)

        # Update indices
        predicted_index += len(actual_MOM)

    return 1 - (residual_sum_squares / total_sum_squares)

# Calculate out-of-sample R-squared for each rolling scheme
r_squared_5_year = calculate_out_of_sample_r_squared(merged_data, predicted_MOM_5_year, residuals_5_year, 5, 5)
r_squared_10_year = calculate_out_of_sample_r_squared(merged_data, predicted_MOM_10_year, residuals_10_year, 10, 5)
r_squared_20_year = calculate_out_of_sample_r_squared(merged_data, predicted_MOM_20_year, residuals_20_year, 20, 5)
r_squared_5_year, r_squared_10_year, r_squared_20_year

(0.6770920604245858, 0.726559645347661, 0.7614872662742465)